In [1]:
from matplotlib import pyplot as plt
from rasterio.mask import mask
import geopandas as gpd
import rasterio as rio
import numpy as np
import mpld3

In [2]:
%matplotlib inline
mpld3.enable_notebook()
plt.rcParams["figure.figsize"] = (9,9)

In [3]:
naip_path = r"naip_clipped_resampled.tif"
chm_path = r"chm_clipped_resampled.tif"
meadow_extent_path = r"../meadow_extent.geojson"

In [24]:
#with rio.open(naip_path) as src:
#    naip_data = src.read()
#    naip_data = np.moveaxis(naip_data, 0, -1)  # Change from (bands, rows, cols) to (rows, cols, bands)
    
with rio.open(chm_path) as src:
    chm_data = src.read()
    chm_profile = src.profile

chm_data = np.squeeze(chm_data)  # Remove single-dimensional entries
#chm_data = np.where(chm_data < 0, 0, chm_data)  # Set negative values to zero

In [5]:
# Read NAIP raster and meadow extent
with rio.open(naip_path) as naip_src:
    naip_crs = naip_src.crs

meadow_gdf = gpd.read_file(meadow_extent_path)

# Reproject meadow extent to NAIP CRS if needed
if meadow_gdf.crs != naip_crs:
    meadow_gdf = meadow_gdf.to_crs(naip_crs)

meadow_geom = meadow_gdf.geometry.values

# Clip NAIP image to meadow extent
with rio.open(naip_path) as src:
    naip_data, naip_clipped_transform = mask(src, meadow_geom, crop=True, nodata=0)
    naip_data = np.transpose(naip_data, (1, 2, 0))  # (rows, cols, bands)

In [6]:
naip_data_float = naip_data.astype(np.float32)
naip_data_float[naip_data_float == 0] = np.nan  # Set zero values to NaN for calculations

In [7]:
naip_data = np.where(naip_data == 0, np.nan, naip_data).astype(np.uint8)  # Set zero values to NaN

C:\Users\jvonnonn\AppData\Local\Temp\1\ipykernel_35548\1220700152.py:1: RuntimeWarning: invalid value encountered in cast
  naip_data = np.where(naip_data == 0, np.nan, naip_data).astype(np.uint8)  # Set zero values to NaN


In [8]:
naip_data.shape, chm_data.shape

((3402, 3658, 3), (3402, 3658))

In [ ]:
brightness = naip_data_float[:, :, 0] + naip_data_float[:, :, 1] + naip_data_float[:, :, 2]
ndvi = (naip_data_float[:, :, 0] - naip_data_float[:, :, 1]) / (naip_data_float[:, :, 0] + naip_data_float[:, :, 1])
ndwi = (naip_data_float[:, :, 1] - naip_data_float[:, :, 0]) / (naip_data_float[:, :, 1] + naip_data_float[:, :, 0])
avg = (naip_data_float[:, :, 0] + naip_data_float[:, :, 1] + naip_data_float[:, :, 2]) / 3
bri_avg = (naip_data_float[:, :, 0] - avg) + (naip_data_float[:, :, 1] - avg) + (naip_data_float[:, :, 2] - avg) / avg

In [69]:
water = (bri_avg <0) & (ndwi > 0)

In [19]:
#bare_mask = (brightness > 320) &(chm_data < 0.5) #& (bri_avg > 29)
bare_mask = (brightness > 320) & (chm_data < 0.5) & ~water & (ndvi < 0.05)  # Adjusted condition to include NDVI threshold
bare_classified = np.zeros(chm_data.shape, dtype=np.uint8)
bare_classified[bare_mask] = 1  # Class 1 for bare
bare_classified[~bare_mask] = 2  # Class 2 for non-bare
bare_classified = np.expand_dims(bare_classified, axis=0)  # Add channel dimension

In [27]:
with rio.open("bare_classified.tif", "w", **chm_profile) as dst:
    dst.write(bare_classified)

## buffer main channel to exclude bare earth within the main channel

In [26]:

import geopandas as gpd
import rasterio
from rasterio.mask import mask
from shapely.geometry import mapping
import numpy as np

def mask_inside_buffer(raster_path, polyline_path, output_path, buffer_distance=10):
    # Step 1: Read the polyline and buffer it
    gdf = gpd.read_file(polyline_path)
    gdf_buffered = gdf.copy()
    gdf_buffered['geometry'] = gdf.geometry.buffer(buffer_distance)

    # Step 2: Open the raster
    with rasterio.open(raster_path) as src:
        raster_data = src.read(1)
        raster_meta = src.meta.copy()
        raster_crs = src.crs

        # Step 3: Reproject buffered geometry to match raster CRS
        if gdf_buffered.crs != raster_crs:
            gdf_buffered = gdf_buffered.to_crs(raster_crs)

        # Step 4: Mask the raster INSIDE the buffer (invert=False means mask inside)
        out_image, out_transform = mask(
            src,
            [mapping(geom) for geom in gdf_buffered.geometry],
            invert=True,  # Keep data OUTSIDE the buffer
            crop=False,
            filled=True,
            nodata=src.nodata
        )

    # Step 5: Update metadata and write output
    raster_meta.update({
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": src.transform,
        "dtype": rasterio.float32
    })

    with rasterio.open(output_path, "w", **raster_meta) as dst:
        dst.write(out_image.astype(rasterio.float32))

    return f"Raster saved to {output_path}, with buffer area masked out."


In [28]:
mask_inside_buffer('bare_classified.tif', r"..\lacey_plong_simp.geojson", 'bare_classified_clipped.tif')

'Raster saved to bare_classified_clipped.tif, with buffer area masked out.'

## Sumary -%cover:  
trees, shrubs, bare earth

In [35]:
with rio.open('bare_classified_clipped.tif') as src:
    bare_classified_clipped = src.read()
    pixel = src.transform.a
    bare_earth_area = np.sum(bare_classified_clipped == 1) * pixel**2

In [49]:
with rio.open("CHM_clipped.tif") as src:
    chm_data = src.read()
    pixel = src.transform.a
    trees = np.sum(chm_data > 5) * pixel**2
    shrubs = np.sum(((chm_data > 0.2) & (chm_data < 5))) * pixel**2
    total_area = np.sum(~np.isnan(chm_data)) * pixel**2

In [71]:
bare_earth_area

np.float64(29035.079999999998)

In [54]:
round(bare_earth_area / total_area * 100, 2)

np.float64(1.67)

In [55]:
round(trees / total_area * 100, 2)

np.float64(2.19)

In [57]:
round(shrubs / total_area * 100, 2)

np.float64(9.45)